In [2]:
import sys
import os

# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if project_root not in sys.path:
    sys.path.insert(0, project_root)



In [3]:
from sentence_transformers import SentenceTransformer

/home/esualp/miniconda3/envs/thesis/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# Download from the 🤗 Hub
model = SentenceTransformer("all-MiniLM-L6-v2")
# Run inference
sentences = [
    'Volksvertreter',
    'Parlamentarier',
    'Oberbürgermeister',
]
embeddings = model.encode(sentences)
print(embeddings.shape)

(3, 384)


In [3]:

# Download from the 🤗 Hub
model = SentenceTransformer("all-MiniLM-L6-v2")
# Run inference
sentences = [
    'Volksvertreter',
    'Parlamentarier',
    'Oberbürgermeister',
]
embeddings = model.encode(sentences, normalize_embeddings=True)
print(embeddings.shape)

(3, 384)


In [4]:
from model import BiEncoder 
import numpy as np

def test_normalization_flag(base_config):
    """Test that the normalize_output flag works correctly."""
    # Test with normalization ON
    base_config.model.proj_dim = None # No projection for simplicity
    model_norm = BiEncoder(base_config.model, base_config.device)
    
    job_titles = ["Data Scientist"]
    job_embs_norm = model_norm.encode_job(job_titles, normalize=True)
    
    # Check that the L2 norm is close to 1
    norm = np.linalg.norm(job_embs_norm, axis=1)
    assert np.allclose(norm, 1.0), "Embeddings should be normalized"

    # Test with normalization OFF
    model_no_norm = BiEncoder(base_config.model, base_config.device)

    job_embs_no_norm = model_no_norm.encode_job(job_titles, normalize=False)

    # Check that the L2 norm is NOT 1
    norm_unnormalized = np.linalg.norm(job_embs_no_norm, axis=1)
    assert not np.allclose(norm_unnormalized, 1.0), "Embeddings should not be normalized"

In [5]:
def base_config(dummy_data_files):
    """A base config that can be modified by other fixtures."""
    pairs_path, esco_path = dummy_data_files
    return Config(
        seed=42,
        device="cpu",
        model=ModelConfig(
            hf_id="sentence-transformers/all-MiniLM-L6-v2",
            proj_dim=None,
            asymmetric=False,
            normalize_output=True,
        ),
        data=DataConfig(pairs_path=pairs_path, esco_titles_path=esco_path),
        infer=InferConfig(batch_size=32, topk=5),
        artifacts=ArtifactsConfig(run_dir="runs/test"),
    )

In [17]:
from src.config import Config, ModelConfig, DataConfig, InferConfig, ArtifactsConfig
import os
import tempfile

# Create dummy files for the config
temp_dir = tempfile.gettempdir()
pairs_path = os.path.join(temp_dir, "dummy_pairs.jsonl")
esco_path = os.path.join(temp_dir, "dummy_esco.jsonl")

with open(pairs_path, "w") as f:
    f.write('{"skill": "python", "job": "Data Scientist"}\n')

with open(esco_path, "w") as f:
    f.write('{"title": "Data Scientist"}\n')

dummy_data_files = (pairs_path, esco_path)

def create_base_config(dummy_data_files):
    """A base config that can be modified by other fixtures."""
    pairs_path, esco_path = dummy_data_files
    return Config(
        seed=42,
        device="cpu",
        model=ModelConfig(
            hf_id="sentence-transformers/all-MiniLM-L6-v2",
            proj_dim=None,
            asymmetric=False,
            normalize_output=False,
        ),
        data=DataConfig(pairs_path=pairs_path, esco_titles_path=esco_path),
        infer=InferConfig(batch_size=32, topk=5),
        artifacts=ArtifactsConfig(run_dir="runs/test"),
    )

base_conf = create_base_config(dummy_data_files)



In [8]:
test_normalization_flag(base_conf)

AssertionError: Embeddings should not be normalized

In [ ]:
from model import BiEncoder 
import numpy as np

def test_normalization_flag(base_config):
    """Test that the normalize_output flag works correctly."""
    # Test with normalization ON
    base_config.model.proj_dim = None # No projection for simplicity
    model_norm = BiEncoder(base_config.model, base_config.device)
    
    job_titles = ["Data Scientist"]
    job_embs_norm = model_norm.encode_job(job_titles, normalize=True)
    
    # Check that the L2 norm is close to 1
    norm = np.linalg.norm(job_embs_norm, axis=1)
    assert np.allclose(norm, 1.0), "Embeddings should be normalized"

    # Test with normalization OFF
    model_no_norm = BiEncoder(base_config.model, base_config.device)

    job_embs_no_norm = model_no_norm.encode_job(job_titles, normalize=False)

    # Check that the L2 norm is NOT 1
    norm_unnormalized = np.linalg.norm(job_embs_no_norm, axis=1)
    assert not np.allclose(norm_unnormalized, 1.0), "Embeddings should not be normalized"

In [18]:

base_conf.model.proj_dim = None # No projection for simplicity
model_norm = BiEncoder(base_conf.model, base_conf.device)

In [19]:
job_titles = ["Data Scientist"]
job_embs_norm = model_norm.encode_job(job_titles, normalize=True)

In [20]:
norm = np.linalg.norm(job_embs_norm, axis=1)
assert np.allclose(norm, 1.0), "Embeddings should be normalized"

In [21]:
model_no_norm = BiEncoder(base_conf.model, base_conf.device)

job_embs_no_norm = model_no_norm.encode_job(job_titles, normalize=False)

In [22]:
model_no_norm

BiEncoder(
  (st_model): SentenceTransformer(
    (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
    (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
    (2): Normalize()
  )
)

In [25]:
# Check that the L2 norm is NOT 1
norm_unnormalized = np.linalg.norm(job_embs_no_norm, axis=1)
assert not np.allclose(norm_unnormalized, 1.0), "Embeddings should not be normalized"

AssertionError: Embeddings should not be normalized

In [28]:
print(SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2"))

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)


In [29]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Drop the last module (Normalize)
model._modules.pop(str(len(model._modules)-1))

print(model)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)


In [31]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load a pretrained model (with Normalize layer in its architecture)
model =  SentenceTransformer("all-mpnet-base-v2")

texts = ["This is a test sentence.", "Another sentence to encode."]

# Encode with normalization
emb_norm = model.encode(texts, normalize_embeddings=True)
# Encode without normalization
emb_raw = model.encode(texts, normalize_embeddings=False)

# Check L2 norms
print("With normalization:")
for i, e in enumerate(emb_norm):
    print(f"  Text {i}: norm = {np.linalg.norm(e):.4f}")

print("\nWithout normalization:")
for i, e in enumerate(emb_raw):
    print(f"  Text {i}: norm = {np.linalg.norm(e):.4f}")


With normalization:
  Text 0: norm = 1.0000
  Text 1: norm = 1.0000

Without normalization:
  Text 0: norm = 1.0000
  Text 1: norm = 1.0000


In [32]:
print(model)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False, 'architecture': 'MPNetModel'})
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)


In [10]:
from sentence_transformers import SentenceTransformer

# Load the JobGTE model from HuggingFace
jobgte_model = SentenceTransformer("pj-mathematician/JobGTE-multilingual-base-v2", trust_remote_code=True)

# Print the model structure
print(jobgte_model)


No sentence-transformers model found with name pj-mathematician/JobGTE-multilingual-base-v2. Creating a new one with mean pooling.


ValueError: Unrecognized model in pj-mathematician/JobGTE-multilingual-base-v2. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: aimv2, aimv2_vision_model, albert, align, altclip, apertus, arcee, aria, aria_text, audio-spectrogram-transformer, autoformer, aya_vision, bamba, bark, bart, beit, bert, bert-generation, big_bird, bigbird_pegasus, biogpt, bit, bitnet, blenderbot, blenderbot-small, blip, blip-2, blip_2_qformer, bloom, blt, bridgetower, bros, camembert, canine, chameleon, chinese_clip, chinese_clip_vision_model, clap, clip, clip_text_model, clip_vision_model, clipseg, clvp, code_llama, codegen, cohere, cohere2, cohere2_vision, colpali, colqwen2, conditional_detr, convbert, convnext, convnextv2, cpmant, csm, ctrl, cvt, d_fine, dab-detr, dac, data2vec-audio, data2vec-text, data2vec-vision, dbrx, deberta, deberta-v2, decision_transformer, deepseek_v2, deepseek_v3, deepseek_vl, deepseek_vl_hybrid, deformable_detr, deit, depth_a

In [9]:
SentenceTransformer()

SentenceTransformer(
  (0): None
)

In [11]:
from sentence_transformers import SentenceTransformer

# Download from the 🤗 Hub
model = SentenceTransformer("pj-mathematician/JobGTE-multilingual-base-v2")
# Run inference
sentences = [
    'Volksvertreter',
    'Parlamentarier',
    'Oberbürgermeister',
]
embeddings = model.encode(sentences)
print(embeddings.shape)
# [3, 768]

# Get the similarity scores for the embeddings
similarities = model.similarity(embeddings, embeddings)
print(similarities.shape)
# [3, 3]


No sentence-transformers model found with name pj-mathematician/JobGTE-multilingual-base-v2. Creating a new one with mean pooling.


ValueError: Unrecognized model in pj-mathematician/JobGTE-multilingual-base-v2. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: aimv2, aimv2_vision_model, albert, align, altclip, apertus, arcee, aria, aria_text, audio-spectrogram-transformer, autoformer, aya_vision, bamba, bark, bart, beit, bert, bert-generation, big_bird, bigbird_pegasus, biogpt, bit, bitnet, blenderbot, blenderbot-small, blip, blip-2, blip_2_qformer, bloom, blt, bridgetower, bros, camembert, canine, chameleon, chinese_clip, chinese_clip_vision_model, clap, clip, clip_text_model, clip_vision_model, clipseg, clvp, code_llama, codegen, cohere, cohere2, cohere2_vision, colpali, colqwen2, conditional_detr, convbert, convnext, convnextv2, cpmant, csm, ctrl, cvt, d_fine, dab-detr, dac, data2vec-audio, data2vec-text, data2vec-vision, dbrx, deberta, deberta-v2, decision_transformer, deepseek_v2, deepseek_v3, deepseek_vl, deepseek_vl_hybrid, deformable_detr, deit, depth_a

In [12]:
from huggingface_hub import snapshot_download
from transformers import AutoTokenizer, AutoModel
from safetensors.torch import load_file
import os, json, torch

# 1) Instantiate the base GTE (has custom code + proper auto_map)
base_id = "Alibaba-NLP/gte-multilingual-base"
tok = AutoTokenizer.from_pretrained(base_id, trust_remote_code=True)
model = AutoModel.from_pretrained(base_id, trust_remote_code=True)

# 2) Download the pj checkpoint locally
repo_id = "pj-mathematician/JobGTE-multilingual-base-v2"
ckpt_dir = snapshot_download(repo_id, allow_patterns=["checkpoint-400/*"])
ckpt = os.path.join(ckpt_dir, "checkpoint-400")

# 3) Load sharded safetensors (handles both single and multi-shard)
index_path = os.path.join(ckpt, "model.safetensors.index.json")
if os.path.exists(index_path):
    index = json.load(open(index_path))
    state_dict = {}
    for shard in sorted(set(index["weight_map"].values())):
        state_dict.update(load_file(os.path.join(ckpt, shard)))
else:
    # fallback if single shard name differs
    state_dict = load_file(os.path.join(ckpt, "model.safetensors"))

# 4) Put weights into the base model
missing, unexpected = model.load_state_dict(state_dict, strict=False)
print("Missing keys:", len(missing), "Unexpected keys:", len(unexpected))

# 5) Use it
inputs = tok(["hello world"], return_tensors="pt", padding=True, truncation=True)
with torch.no_grad():
    out = model(**inputs).last_hidden_state
emb = (out * inputs["attention_mask"].unsqueeze(-1)).sum(1) / inputs["attention_mask"].sum(1)
print(emb.shape)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

configuration.py: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

trainer_state.json: 0.00B [00:00, ?B/s]

checkpoint-400/training_args.bin:   0%|          | 0.00/5.62k [00:00<?, ?B/s]

: 

In [1]:
import torch
import numpy as np
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import batch_to_device, cos_sim

# Load the model
model = SentenceTransformer("TechWolf/JobBERT-v2")

def encode_batch(jobbert_model, texts):
    features = jobbert_model.tokenize(texts)
    features = batch_to_device(features, jobbert_model.device)
    features["text_keys"] = ["anchor"]
    with torch.no_grad():
        out_features = jobbert_model.forward(features)
    return out_features["sentence_embedding"].cpu().numpy()

def encode(jobbert_model, texts, batch_size: int = 8):
    # Sort texts by length and keep track of original indices
    sorted_indices = np.argsort([len(text) for text in texts])
    sorted_texts = [texts[i] for i in sorted_indices]
    
    embeddings = []
    
    # Encode in batches
    for i in tqdm(range(0, len(sorted_texts), batch_size)):
        batch = sorted_texts[i:i+batch_size]
        embeddings.append(encode_batch(jobbert_model, batch))
    
    # Concatenate embeddings and reorder to original indices
    sorted_embeddings = np.concatenate(embeddings)
    original_order = np.argsort(sorted_indices)
    return sorted_embeddings[original_order]

# Example usage
job_titles = [
    'Software Engineer',
    'Senior Software Developer',
    'Product Manager',
    'Data Scientist'
]

# Get embeddings
embeddings = encode(model, job_titles)

# Calculate cosine similarity matrix
similarities = cos_sim(embeddings, embeddings)
print(similarities)


  0%|          | 0/1 [00:00<?, ?it/s]

tensor([[1.0000, 0.8723, 0.4821, 0.5447],
        [0.8723, 1.0000, 0.4822, 0.5019],
        [0.4821, 0.4822, 1.0000, 0.4328],
        [0.5447, 0.5019, 0.4328, 1.0000]])


In [2]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False, 'architecture': 'MPNetModel'})
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Router(
    (sub_modules): ModuleDict(
      (anchor): Sequential(
        (0): Dense({'in_features': 768, 'out_features': 1024, 'bias': True, 'activation_function': 'torch.nn.modules.activation.Tanh'})
      )
      (positive): Sequential(
        (0): Dense({'in_features': 768, 'out_features': 1024, 'bias': True, 'activation_function': 'torch.nn.modules.activation.Tanh'})
      )
    )
  )
)

In [5]:
from sentence_transformers import util

full = model

skill_phrases = [
    "Python",
    "Data management",
    "Teamwork",
    "SQL"
]
# full is your loaded JobBERT model
full = SentenceTransformer("TechWolf/JobBERT-v2")  # or your local path

# Create asymmetric encoders using the proper 'modules' argument
title_encoder = SentenceTransformer(modules=[
    full[0],                        # Transformer
    full[1],                        # Pooling
    full[2].sub_modules['anchor']   # Anchor head (Dense)
])

skill_encoder = SentenceTransformer(modules=[
    full[0],                        # Transformer
    full[1],                        # Pooling
    full[2].sub_modules['positive'] # Positive head (Dense)
])

# Encode your texts
E_job   = title_encoder.encode(job_titles,   normalize_embeddings=True)
E_skill = skill_encoder.encode(skill_phrases, normalize_embeddings=True)
scores = util.cos_sim(E_job, E_skill)


In [6]:
scores

tensor([[-0.0388, -0.0049,  0.0389,  0.0165],
        [-0.0348, -0.0025,  0.0402,  0.0236],
        [-0.0155,  0.0316,  0.0120, -0.0080],
        [-0.0177, -0.0262,  0.0507, -0.0017]])

In [7]:
# Encode your texts
penultimate = SentenceTransformer(modules=[full[0], full[1]])
E_job   = penultimate.encode(job_titles,   normalize_embeddings=True)
E_skill = penultimate.encode(skill_phrases, normalize_embeddings=True)
scores = util.cos_sim(E_job, E_skill)

In [8]:
scores

tensor([[0.3860, 0.3177, 0.1637, 0.3031],
        [0.3554, 0.2970, 0.0497, 0.3300],
        [0.1582, 0.3106, 0.0528, 0.1499],
        [0.4546, 0.4501, 0.0885, 0.3586]])

In [9]:
job_titles

['Software Engineer',
 'Senior Software Developer',
 'Product Manager',
 'Data Scientist']

In [11]:
skill_phrases

['Python', 'Data management', 'Teamwork', 'SQL']

# Fetching model issue

In [ ]:
from sentence_transformers import SentenceTransformer

# Download from the 🤗 Hub
model = SentenceTransformer("pj-mathematician/JobSkillBGE-large-en-v1.5/checkpoint-4400")
# Run inference
sentences = [
    'For roles such as import/export manager, graduate export manager, senior export manager, and other related positions in meat and meat products, the key skills include a strong understanding of international trade regulations, meat product knowledge, customs compliance, and excellent negotiation and communication skills to manage global supply chains effectively. Additionally, proficiency in relevant trade software and languages can be highly beneficial.',
    'Job roles such as Performance Analyst, Quality Assurance Engineer, and Test Manager require skills in conducting performance measurement and organizing or managing conversion testing to ensure software and systems meet performance standards and function correctly in real-world scenarios.',
    'Job roles that require skills such as managing staff, coordinating employees, and performing HR activities include Human Resources Managers, Team Leaders, Supervisors, and Department Heads, all of whom are responsible for overseeing personnel, implementing HR policies, and ensuring efficient team operations.',
]
embeddings = model.encode(sentences)
print(embeddings.shape)
# [3, 1024]

# Get the similarity scores for the embeddings
similarities = model.similarity(embeddings, embeddings)
print(similarities.shape)
# [3, 3]


FileNotFoundError: Path pj-mathematician/JobSkillBGE-large-en-v1.5/4400 not found

In [10]:
import os
from sentence_transformers import SentenceTransformer
from huggingface_hub import snapshot_download

# 1. Define the repo and the specific subfolder you want
repo_id = "pj-mathematician/JobSkillBGE-large-en-v1.5"
checkpoint_subfolder = "checkpoint-4480"

# 2. Download *only* the contents of that subfolder
# This downloads the files to your local cache and returns the path to the main snapshot
snapshot_path = snapshot_download(
    repo_id=repo_id,
    allow_patterns=[f"{checkpoint_subfolder}/*"]  # This downloads only the checkpoint files
)

# 3. Create the full local path to the model files
# The files are inside the checkpoint folder within the snapshot
model_path = os.path.join(snapshot_path, checkpoint_subfolder)

# 4. Now, load the model from the *local path*
print(f"Loading model from local path: {model_path}")
model = SentenceTransformer(model_path)

# 5. Run inference (your original code)
sentences = [
    'For roles such as import/export manager, graduate export manager...',
    'Job roles such as Performance Analyst, Quality Assurance Engineer...',
    'Job roles that require skills such as managing staff...',
]
embeddings = model.encode(sentences)
print(embeddings.shape)

Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

checkpoint-4480/model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

checkpoint-4480/rng_state.pth:   0%|          | 0.00/15.9k [00:00<?, ?B/s]

checkpoint-4480/optimizer.pt:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

...

checkpoint-4480/training_args.bin:   0%|          | 0.00/5.56k [00:00<?, ?B/s]

trainer_state.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Cancellation requested; stopping current tasks.


KeyboardInterrupt: 

In [ ]:
model_path = "TechWolf/JobBERT-v2"
model = SentenceTransformer(model_path)
